<a href="https://colab.research.google.com/github/ikszn/V2G-Potential/blob/main/V2GPotentialv11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# V2G SCENARIO ANALYSIS: ACADEMIC JUPYTER NOTEBOOK STRUCTURE


Vehicle-to-Grid (V2G) Scenario Analysis Framework
Complete Academic Implementation Starting from Raw EV Usage Data

This notebook provides comprehensive analysis of V2G potential through:
1. Raw data loading and validation
2. Data preprocessing and V2G potential calculation
3. Hourly aggregation and feature engineering
4. Exploratory Data Analysis of usage patterns
5. Market pricing analysis (wholesale vs imbalance markets)  
6. Integrated scenario modeling
7. Academic results presentation and discussion


Author: Ikram Ahmed
Institution: UCL
Date: 13/09/25


# SECTION 0: IMPORT LIBRARIES


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from typing import Dict, Any, Tuple, List
from datetime import datetime, timedelta
import os

# Configure display options for academic presentation
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')

print("📊 V2G Scenario Analysis Framework - Complete Academic Pipeline")
print("="*80)

📊 V2G Scenario Analysis Framework - Complete Academic Pipeline


# SECTION 1: RAW DATA LOADING AND INITIAL VALIDATION


In [3]:
def load_and_validate_raw_data(raw_df: pd.DataFrame) -> Dict[str, Any]:
    """
    Load and validate raw EV usage data with automatic column mapping for your format
    """

    print("\n1️⃣ RAW DATA LOADING AND VALIDATION")
    print("-" * 50)

    # Create working copy to avoid modifying original
    df = raw_df.copy()

    # Basic dataset structure
    n_rows, n_cols = df.shape

    print(f"📊 Raw Dataset Dimensions: {n_rows:,} records × {n_cols} columns")
    print(f"📋 Actual columns in your data:")
    for i, col in enumerate(df.columns):
        print(f"   {i+1}. '{col}'")

    # Column mapping for your specific data format
    column_mapping = {
        'ParticipantID': 'vehicle_id',
        'BatteryChargeStartDate': 'start_time',
        'BatteryChargeStopDate': 'end_time',
        'Starting SoC (of 12)': 'start_soc_raw',
        'Ending SoC (of 12)': 'end_soc_raw'
    }

    print(f"\n🔄 Column Mapping Attempt:")

    # Check exactly which columns exist and which don't
    successful_mappings = {}
    failed_mappings = {}

    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df[new_name] = df[old_name]
            successful_mappings[old_name] = new_name
            print(f"   ✅ '{old_name}' → '{new_name}'")
        else:
            failed_mappings[old_name] = new_name
            print(f"   ❌ '{old_name}' NOT FOUND")

    # If any mappings failed, show what we have vs what we expected
    if failed_mappings:
        print(f"\n⚠️  COLUMN MAPPING ISSUES:")
        print(f"   Failed to map: {list(failed_mappings.keys())}")
        print(f"   Your actual columns: {list(df.columns)}")

        # Try fuzzy matching to suggest corrections
        print(f"\n🔍 Possible matches:")
        for expected_col in failed_mappings.keys():
            for actual_col in df.columns:
                if expected_col.lower() in actual_col.lower() or actual_col.lower() in expected_col.lower():
                    print(f"   '{expected_col}' might be '{actual_col}'")

        raise ValueError(f"Column mapping failed. Please check your column names match exactly.")

    # Verify all required columns now exist
    required_columns = ['vehicle_id', 'start_time', 'end_time', 'start_soc_raw', 'end_soc_raw']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        print(f"❌ Missing required columns after mapping: {missing_columns}")
        print(f"Available columns: {list(df.columns)}")
        raise ValueError(f"Required columns missing: {missing_columns}")

    print(f"✅ All required columns successfully mapped!")

    # Get vehicle count after mapping
    n_vehicles = df['vehicle_id'].nunique()
    print(f"🚗 Vehicle Sample: {n_vehicles:,} unique participants")

        # Column mapping for your specific data format
    column_mapping = {
        'ParticipantID': 'vehicle_id',
        'BatteryChargeStartDate': 'start_time',
        'BatteryChargeStopDate': 'end_time',
        'Starting SoC (of 12)': 'start_soc_raw',
        'Ending SoC (of 12)': 'end_soc_raw'
    }

    print(f"\n🔄 Column Mapping and Standardization:")

    # Check if we have the expected columns for your data format
    your_columns = list(column_mapping.keys())
    missing_columns = [col for col in your_columns if col not in df.columns]

    if missing_columns:
        print(f"   ❌ Missing expected columns: {missing_columns}")
        print(f"   Available columns: {list(df.columns)}")
        raise ValueError(f"Expected columns from your data format not found: {missing_columns}")

    # Apply column mapping
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df[new_name] = df[old_name]
            print(f"   ✓ {old_name} → {new_name}")

    # Get vehicle count after mapping
    n_vehicles = df['vehicle_id'].nunique()
    print(f"🚗 Vehicle Sample: {n_vehicles:,} unique participants")

    # Data type validation and conversion
    print(f"\n🔍 Data Type Validation and Conversion:")

    # Convert timestamps
    for time_col in ['start_time', 'end_time']:
        if not pd.api.types.is_datetime64_any_dtype(df[time_col]):
            df[time_col] = pd.to_datetime(df[time_col])
            print(f"   ✓ Converted {time_col} to datetime")

    # Convert SoC from "of 12" to percentage
    print(f"\n⚡ SoC Conversion (of 12 → percentage):")

    for soc_col in ['start_soc_raw', 'end_soc_raw']:
        # Convert to numeric first
        df[soc_col] = pd.to_numeric(df[soc_col], errors='coerce')
        invalid_count = df[soc_col].isna().sum()
        if invalid_count > 0:
            print(f"   ⚠️ {soc_col}: {invalid_count} invalid values found")

    # Convert to percentage (assuming max is 12)
    df['start_soc'] = (df['start_soc_raw'] / 12.0) * 100
    df['end_soc'] = (df['end_soc_raw'] / 12.0) * 100

    print(f"   ✓ Converted SoC values to percentage scale:")
    print(f"     Start SoC range: {df['start_soc'].min():.1f}% - {df['start_soc'].max():.1f}%")
    print(f"     End SoC range: {df['end_soc'].min():.1f}% - {df['end_soc'].max():.1f}%")

    # Validate SoC conversion makes sense
    if df['start_soc'].max() > 100 or df['end_soc'].max() > 100:
        print(f"   ⚠️ Warning: Some SoC values exceed 100% after conversion")

    if df['start_soc'].min() < 0 or df['end_soc'].min() < 0:
        print(f"   ⚠️ Warning: Some SoC values are negative after conversion")

    # Temporal coverage analysis
    date_min = df['start_time'].min()
    date_max = df['end_time'].max()
    total_days = (date_max - date_min).days + 1

    print(f"\n📅 Temporal Coverage:")
    print(f"   Period: {date_min.strftime('%Y-%m-%d')} to {date_max.strftime('%Y-%m-%d')}")
    print(f"   Duration: {total_days} days")

    # Data quality assessment
    print(f"\n📋 Data Quality Assessment:")

    # Check for duplicate records
    duplicates = df.duplicated().sum()
    print(f"   Duplicate records: {duplicates:,} ({duplicates/len(df)*100:.1f}%)")

    # Check for invalid time sequences
    invalid_times = (df['end_time'] <= df['start_time']).sum()
    print(f"   Invalid time sequences: {invalid_times:,}")

    # Check for impossible SoC values (after conversion to %)
    invalid_soc = ((df['start_soc'] < 0) | (df['start_soc'] > 100) |
                   (df['end_soc'] < 0) | (df['end_soc'] > 100)).sum()
    print(f"   Invalid SoC values (after % conversion): {invalid_soc:,}")

    # Check for backwards SoC (discharge sessions)
    discharge_sessions = (df['end_soc'] < df['start_soc']).sum()
    charging_sessions = (df['end_soc'] > df['start_soc']).sum()

    print(f"   Charging sessions (SoC increase): {charging_sessions:,} ({charging_sessions/len(df)*100:.1f}%)")
    print(f"   Discharge sessions (SoC decrease): {discharge_sessions:,} ({discharge_sessions/len(df)*100:.1f}%)")

    # Missing data analysis
    missing_data = df.isnull().sum()
    total_missing = missing_data.sum()
    print(f"   Total missing values: {total_missing:,} ({total_missing/(len(df)*len(df.columns))*100:.1f}%)")

    if total_missing > 0:
        print(f"   Missing by column:")
        for col, missing_count in missing_data[missing_data > 0].items():
            print(f"     {col}: {missing_count:,} ({missing_count/len(df)*100:.1f}%)")

    # Vehicle-level statistics
    sessions_per_vehicle = df.groupby('vehicle_id').size()
    print(f"\n🚗 Participant-Level Statistics:")
    print(f"   Sessions per participant - Mean: {sessions_per_vehicle.mean():.1f}, Median: {sessions_per_vehicle.median():.1f}")
    print(f"   Range: {sessions_per_vehicle.min()} - {sessions_per_vehicle.max()} sessions")

    # Session duration analysis
    df['session_duration_hours'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 3600
    duration_stats = df['session_duration_hours'].describe()
    print(f"\n⏱️ Session Duration Analysis:")
    print(f"   Mean duration: {duration_stats['mean']:.1f} hours")
    print(f"   Range: {duration_stats['min']:.1f} - {duration_stats['max']:.1f} hours")

    # SoC behavior analysis
    df['soc_change'] = df['end_soc'] - df['start_soc']
    soc_change_stats = df['soc_change'].describe()

    print(f"\n🔋 SoC Change Analysis:")
    print(f"   Mean SoC change: {soc_change_stats['mean']:.1f} percentage points")
    print(f"   SoC change range: {soc_change_stats['min']:.1f} to {soc_change_stats['max']:.1f} pp")

    # Data cleaning recommendations
    print(f"\n🧹 Data Cleaning Recommendations:")

    recommendations = []

    if invalid_times > 0:
        recommendations.append(f"Remove {invalid_times} records with invalid time sequences")

    if invalid_soc > 0:
        recommendations.append(f"Review {invalid_soc} records with impossible SoC values")

    if discharge_sessions > charging_sessions * 0.1:  # More than 10% discharge sessions
        recommendations.append(f"Consider filtering {discharge_sessions} discharge sessions for V2G analysis")

    if duration_stats['max'] > 48:  # Sessions longer than 48 hours
        long_sessions = (df['session_duration_hours'] > 48).sum()
        recommendations.append(f"Review {long_sessions} unusually long sessions (>48 hours)")

    for i, rec in enumerate(recommendations, 1):
        print(f"   {i}. {rec}")

    if not recommendations:
        print(f"   ✅ Data appears clean and ready for V2G analysis")

    return {
        'n_records': len(df),
        'n_vehicles': n_vehicles,
        'n_days': total_days,
        'date_range': (date_min, date_max),
        'data_quality': {
            'duplicates': duplicates,
            'invalid_times': invalid_times,
            'invalid_soc': invalid_soc,
            'total_missing': total_missing,
            'charging_sessions': charging_sessions,
            'discharge_sessions': discharge_sessions
        },
        'sessions_per_vehicle': sessions_per_vehicle.describe().to_dict(),
        'duration_stats': duration_stats.to_dict(),
        'soc_stats': soc_change_stats.to_dict(),
        'processed_dataframe': df  # Return the processed dataframe with standardized columns
    }


# SECTION 2: DATA PREPROCESSING AND V2G POTENTIAL CALCULATION


In [4]:
def calculate_v2g_potential(raw_df: pd.DataFrame, battery_capacity_kwh: float = 40.0) -> pd.DataFrame:
    """
    Calculate V2G export potential from raw EV usage data

    This function transforms raw session data into V2G export calculations
    by analyzing charging sessions and determining exportable energy

    Parameters:
    -----------
    raw_df : pd.DataFrame
        Raw EV usage data
    battery_capacity_kwh : float
        Assumed battery capacity in kWh (default: 40 kWh for older EVs)

    Returns:
    --------
    pd.DataFrame with V2G potential calculations added
    """

    print("\n2️⃣ V2G POTENTIAL CALCULATION")
    print("-" * 50)

    # Create working copy
    df = raw_df.copy()

    # Calculate energy metrics
    print(f"🔋 Calculating energy metrics with {battery_capacity_kwh} kWh battery assumption")

    # Energy charged during session (based on SoC change)
    df['soc_change'] = df['end_soc'] - df['start_soc']
    df['energy_charged_kwh'] = (df['soc_change'] / 100) * battery_capacity_kwh

    # Only consider sessions where energy was added (charging sessions)
    charging_sessions = df[df['energy_charged_kwh'] > 0].copy()

    print(f"   Total sessions: {len(df):,}")
    print(f"   Charging sessions: {len(charging_sessions):,} ({len(charging_sessions)/len(df)*100:.1f}%)")

    # Calculate V2G export potential with different reserve levels
    reserve_levels = [20, 40, 60]  # Percentage of battery to keep in reserve

    print(f"\n⚡ Calculating V2G export potential at different reserve levels:")

    for reserve_pct in reserve_levels:
        # Available energy for export = energy above reserve level
        charging_sessions[f'available_energy_reserve_{reserve_pct}'] = np.maximum(
            0,
            (charging_sessions['end_soc'] - reserve_pct) / 100 * battery_capacity_kwh
        )

        # Conservative V2G export assumption: export 80% of available energy
        # (accounts for charging losses, user behavior, system constraints)
        export_efficiency = 0.80
        charging_sessions[f'v2g_export_potential_reserve_{reserve_pct}'] = (
            charging_sessions[f'available_energy_reserve_{reserve_pct}'] * export_efficiency
        )

        # Statistics for this reserve level
        total_potential = charging_sessions[f'v2g_export_potential_reserve_{reserve_pct}'].sum()
        sessions_with_export = (charging_sessions[f'v2g_export_potential_reserve_{reserve_pct}'] > 0).sum()

        print(f"   {reserve_pct}% reserve: {total_potential:.1f} kWh potential, {sessions_with_export:,} sessions ({sessions_with_export/len(charging_sessions)*100:.1f}%)")

    # Add temporal features for hourly analysis
    charging_sessions['start_hour'] = charging_sessions['start_time'].dt.hour
    charging_sessions['start_dow'] = charging_sessions['start_time'].dt.dayofweek
    charging_sessions['start_date'] = charging_sessions['start_time'].dt.date

    # Calculate session-level participation flags
    for reserve_pct in reserve_levels:
        charging_sessions[f'can_participate_reserve_{reserve_pct}'] = (
            charging_sessions[f'v2g_export_potential_reserve_{reserve_pct}'] > 0.1  # Minimum 0.1 kWh threshold
        )

    print(f"\n📊 V2G Participation Analysis:")
    for reserve_pct in reserve_levels:
        participants = charging_sessions[f'can_participate_reserve_{reserve_pct}'].sum()
        print(f"   {reserve_pct}% reserve: {participants:,} sessions can participate ({participants/len(charging_sessions)*100:.1f}%)")

    return charging_sessions

def create_hourly_aggregation(charging_sessions_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate session-based data into hourly time series for V2G analysis

    Creates hourly records showing V2G export potential across different
    reserve levels and participation scenarios

    Parameters:
    -----------
    charging_sessions_df : pd.DataFrame
        Processed charging sessions with V2G calculations

    Returns:
    --------
    pd.DataFrame with hourly V2G export aggregations
    """

    print("\n3️⃣ HOURLY AGGREGATION AND TIME SERIES CREATION")
    print("-" * 50)

    # Get temporal range
    start_date = charging_sessions_df['start_time'].min().floor('H')
    end_date = charging_sessions_df['start_time'].max().ceil('H')

    print(f"📅 Creating hourly time series from {start_date} to {end_date}")

    # Create complete hourly index
    hourly_index = pd.date_range(start=start_date, end=end_date, freq='H')

    # Get all vehicles
    all_vehicles = charging_sessions_df['vehicle_id'].unique()

    print(f"🚗 Processing {len(all_vehicles):,} vehicles across {len(hourly_index):,} hours")

    # Create base hourly dataframe
    hourly_records = []

    for vehicle_id in all_vehicles:
        vehicle_sessions = charging_sessions_df[charging_sessions_df['vehicle_id'] == vehicle_id]

        for timestamp in hourly_index:
            # Find sessions that overlap with this hour
            hour_start = timestamp
            hour_end = timestamp + timedelta(hours=1)

            # Sessions that started in this hour (simplified approach)
            hour_sessions = vehicle_sessions[
                (vehicle_sessions['start_time'] >= hour_start) &
                (vehicle_sessions['start_time'] < hour_end)
            ]

            # Aggregate V2G potential for this hour
            record = {
                'vehicle_id': vehicle_id,
                'timestamp': timestamp,
                'hour': timestamp.hour,
                'day_of_week': timestamp.dayofweek,
                'date': timestamp.date(),
                'n_sessions': len(hour_sessions)
            }

            # Add export potential for each reserve level
            reserve_levels = [20, 40, 60]
            for reserve_pct in reserve_levels:
                export_col = f'v2g_export_potential_reserve_{reserve_pct}'
                if export_col in hour_sessions.columns:
                    record[f'export_kwh_reserve_{reserve_pct}'] = hour_sessions[export_col].sum()
                else:
                    record[f'export_kwh_reserve_{reserve_pct}'] = 0.0

            # Add participation flags
            for reserve_pct in reserve_levels:
                participate_col = f'can_participate_reserve_{reserve_pct}'
                if participate_col in hour_sessions.columns:
                    record[f'can_participate_reserve_{reserve_pct}'] = hour_sessions[participate_col].any()
                else:
                    record[f'can_participate_reserve_{reserve_pct}'] = False

            hourly_records.append(record)

    # Create hourly dataframe
    hourly_df = pd.DataFrame(hourly_records)

    # Data quality summary
    total_export_40 = hourly_df['export_kwh_reserve_40'].sum()
    active_hours = (hourly_df['export_kwh_reserve_40'] > 0).sum()
    active_vehicles = hourly_df[hourly_df['export_kwh_reserve_40'] > 0]['vehicle_id'].nunique()

    print(f"\n📊 Hourly Aggregation Summary:")
    print(f"   Total records created: {len(hourly_df):,}")
    print(f"   Active hours (40% reserve): {active_hours:,} ({active_hours/len(hourly_df)*100:.1f}%)")
    print(f"   Total V2G potential (40% reserve): {total_export_40:.1f} kWh")
    print(f"   Vehicles with V2G activity: {active_vehicles:,}/{len(all_vehicles):,}")

    # Temporal pattern summary
    hourly_patterns = hourly_df.groupby('hour')['export_kwh_reserve_40'].sum()
    peak_hour = hourly_patterns.idxmax()
    peak_export = hourly_patterns.max()

    print(f"   Peak V2G hour: {peak_hour}:00 ({peak_export:.1f} kWh)")

    return hourly_df

# SECTION 3: COMPREHENSIVE EXPLORATORY DATA ANALYSIS


In [5]:
def perform_comprehensive_eda(hourly_df: pd.DataFrame, raw_validation: Dict) -> Dict[str, Any]:
    """
    Comprehensive Exploratory Data Analysis of processed V2G data

    Analyzes patterns in the processed hourly data to understand:
    - Vehicle participation patterns
    - Temporal usage trends
    - Energy export distributions
    - User behavior segmentation
    """

    print("\n4️⃣ EXPLORATORY DATA ANALYSIS OF PROCESSED DATA")
    print("-" * 50)

    eda_results = {}
    main_export_col = 'export_kwh_reserve_40'  # Use 40% reserve as primary metric

    # --- 3.1 Vehicle Participation Analysis ---
    print("\n📋 4.1 Vehicle Participation Analysis")

    # Calculate vehicle-level aggregations
    vehicle_totals = hourly_df.groupby('vehicle_id')[main_export_col].agg([
        'count', 'sum', 'mean', 'std', 'max'
    ]).round(2)

    # Participation classification
    vehicles_with_export = (vehicle_totals['sum'] > 0).sum()
    participation_rate = vehicles_with_export / len(vehicle_totals)

    print(f"   Active V2G Participants: {vehicles_with_export}/{len(vehicle_totals)} vehicles ({participation_rate:.1%})")

    # Export distribution analysis
    active_vehicles = vehicle_totals[vehicle_totals['sum'] > 0]

    if len(active_vehicles) > 0:
        export_stats = active_vehicles['sum'].describe()
        print(f"\n   Export Distribution (Active Vehicles Only):")
        print(f"     Mean export: {export_stats['mean']:.1f} kWh/vehicle (study period)")
        print(f"     Median export: {export_stats['50%']:.1f} kWh/vehicle")
        print(f"     Std deviation: {export_stats['std']:.1f} kWh")
        print(f"     Range: {export_stats['min']:.1f} - {export_stats['max']:.1f} kWh")

        # Annualize the export values
        study_days = raw_validation['n_days']
        annual_factor = 365.25 / study_days
        annual_export_stats = export_stats * annual_factor

        print(f"\n   Annualized Export Estimates:")
        print(f"     Mean annual export: {annual_export_stats['mean']:.0f} kWh/vehicle/year")
        print(f"     Median annual export: {annual_export_stats['50%']:.0f} kWh/vehicle/year")

        # User segmentation
        q75 = export_stats['75%']
        q25 = export_stats['25%']
        heavy_users = (active_vehicles['sum'] >= q75).sum()
        light_users = (active_vehicles['sum'] <= q25).sum()

        print(f"\n   User Segmentation:")
        print(f"     Heavy users (≥75th percentile): {heavy_users} vehicles (≥{q75:.1f} kWh)")
        print(f"     Light users (≤25th percentile): {light_users} vehicles (≤{q25:.1f} kWh)")

    eda_results['participation'] = {
        'total_vehicles': len(vehicle_totals),
        'active_vehicles': vehicles_with_export,
        'participation_rate': participation_rate,
        'export_distribution': export_stats.to_dict() if len(active_vehicles) > 0 else None,
        'annual_export_per_vehicle': annual_export_stats['mean'] if len(active_vehicles) > 0 else 0,
        'annual_export_per_active_vehicle': annual_export_stats['mean'] * len(vehicle_totals) / vehicles_with_export if vehicles_with_export > 0 else 0
    }

    # --- 3.2 Temporal Pattern Analysis ---
    print("\n📅 4.2 Temporal Pattern Analysis")

    # Hourly patterns
    hourly_patterns = hourly_df.groupby('hour')[main_export_col].agg([
        'sum', 'mean', 'count'
    ]).round(2)

    peak_hour = hourly_patterns['sum'].idxmax()
    peak_export = hourly_patterns['sum'].max()
    off_peak_hour = hourly_patterns['sum'].idxmin()

    print(f"   Hourly Patterns:")
    print(f"     Peak export hour: {peak_hour}:00 ({peak_export:.1f} kWh total)")
    print(f"     Lowest export hour: {off_peak_hour}:00 ({hourly_patterns['sum'].min():.1f} kWh total)")

    # Peak vs off-peak analysis
    peak_hours = [17, 18, 19, 20]  # 5-8 PM
    off_peak_hours = [1, 2, 3, 4, 5]  # 1-5 AM

    peak_export_total = hourly_df[hourly_df['hour'].isin(peak_hours)][main_export_col].sum()
    off_peak_export_total = hourly_df[hourly_df['hour'].isin(off_peak_hours)][main_export_col].sum()
    peak_ratio = peak_export_total / max(off_peak_export_total, 1)

    print(f"     Peak period export (5-8 PM): {peak_export_total:.1f} kWh")
    print(f"     Off-peak export (1-5 AM): {off_peak_export_total:.1f} kWh")
    print(f"     Peak vs off-peak ratio: {peak_ratio:.1f}:1")

    # Weekly patterns
    weekly_patterns = hourly_df.groupby('day_of_week')[main_export_col].sum()
    weekday_avg = weekly_patterns[:5].mean()  # Mon-Fri
    weekend_avg = weekly_patterns[5:].mean()  # Sat-Sun
    weekend_ratio = weekend_avg / max(weekday_avg, 1)

    print(f"\n   Weekly Patterns:")
    print(f"     Weekday average: {weekday_avg:.1f} kWh/day")
    print(f"     Weekend average: {weekend_avg:.1f} kWh/day")
    print(f"     Weekend vs weekday ratio: {weekend_ratio:.2f}:1")

    eda_results['temporal'] = {
        'peak_hour': peak_hour,
        'peak_export': peak_export,
        'peak_vs_offpeak_ratio': peak_ratio,
        'weekend_vs_weekday_ratio': weekend_ratio,
        'hourly_distribution': hourly_patterns['sum'].to_dict()
    }

    # --- 3.3 Reserve Level Comparison ---
    print("\n🔋 4.3 Reserve Level Impact Analysis")

    reserve_comparison = {}
    for reserve_pct in [20, 40, 60]:
        export_col = f'export_kwh_reserve_{reserve_pct}'
        total_export = hourly_df[export_col].sum()
        active_hours = (hourly_df[export_col] > 0).sum()
        vehicles_participating = hourly_df[hourly_df[export_col] > 0]['vehicle_id'].nunique()

        reserve_comparison[reserve_pct] = {
            'total_export': total_export,
            'active_hours': active_hours,
            'vehicles_participating': vehicles_participating
        }

        print(f"   {reserve_pct}% Reserve Level:")
        print(f"     Total export: {total_export:.1f} kWh")
        print(f"     Active hours: {active_hours:,}")
        print(f"     Participating vehicles: {vehicles_participating}")

    # Calculate reserve level impacts
    baseline_export = reserve_comparison[40]['total_export']  # 40% as baseline
    for reserve_pct in [20, 60]:
        impact = (reserve_comparison[reserve_pct]['total_export'] - baseline_export) / baseline_export * 100
        print(f"     {reserve_pct}% vs 40% reserve: {impact:+.1f}% export difference")

    eda_results['reserve_analysis'] = reserve_comparison

    # --- 3.4 Energy Economics Baseline ---
    print("\n💰 4.4 Energy Economics Baseline")

    total_export = hourly_df[main_export_col].sum()
    study_days = raw_validation['n_days']
    annual_export_total = total_export * (365.25 / study_days)

    annual_export_per_vehicle = annual_export_total / raw_validation['n_vehicles']
    annual_export_per_active_vehicle = annual_export_total / max(vehicles_with_export, 1)

    print(f"   Energy Export Summary:")
    print(f"     Total export (study period): {total_export:.1f} kWh")
    print(f"     Annualized total export: {annual_export_total:.0f} kWh/year")
    print(f"     Annual export per vehicle (all): {annual_export_per_vehicle:.1f} kWh/year")
    print(f"     Annual export per active vehicle: {annual_export_per_active_vehicle:.1f} kWh/year")

    # Grid impact estimation
    avg_hourly_export = total_export / (study_days * 24)
    peak_hourly_export = hourly_df.groupby('timestamp')[main_export_col].sum().max()
    capacity_factor = avg_hourly_export / max(peak_hourly_export, 1)

    print(f"\n   Grid Impact Indicators:")
    print(f"     Average hourly export: {avg_hourly_export:.2f} kWh/hour")
    print(f"     Peak hourly export: {peak_hourly_export:.1f} kWh/hour")
    print(f"     Export capacity factor: {capacity_factor:.2%}")

    eda_results['economics'] = {
        'total_export_kwh': total_export,
        'annual_export_per_vehicle': annual_export_per_vehicle,
        'annual_export_per_active_vehicle': annual_export_per_active_vehicle,
        'peak_hourly_export': peak_hourly_export,
        'capacity_factor': capacity_factor
    }

    # --- 3.5 Data Processing Quality Assessment ---
    print("\n📊 4.5 Data Processing Quality Assessment")

    # Compare raw sessions to processed hours
    raw_sessions = raw_validation['n_records']
    processed_hours = len(hourly_df)
    active_hours = (hourly_df[main_export_col] > 0).sum()

    print(f"   Data Transformation Summary:")
    print(f"     Raw sessions processed: {raw_sessions:,}")
    print(f"     Hourly records created: {processed_hours:,}")
    print(f"     Hours with V2G activity: {active_hours:,} ({active_hours/processed_hours*100:.1f}%)")

    # Data retention analysis
    vehicles_in_raw = raw_validation['n_vehicles']
    vehicles_in_processed = hourly_df['vehicle_id'].nunique()
    retention_rate = vehicles_in_processed / vehicles_in_raw

    print(f"     Vehicle retention: {vehicles_in_processed}/{vehicles_in_raw} ({retention_rate:.1%})")

    # --- 3.6 Key Insights Summary ---
    print("\n💡 4.6 Key Data Insights")
    print("-" * 30)

    insights = []

    if participation_rate < 0.3:
        insights.append(f"Low participation rate ({participation_rate:.1%}) suggests V2G requires specific conditions")
    elif participation_rate > 0.7:
        insights.append(f"High participation rate ({participation_rate:.1%}) indicates strong V2G potential")
    else:
        insights.append(f"Moderate participation rate ({participation_rate:.1%}) shows selective V2G usage")

    if peak_ratio > 2:
        insights.append(f"Strong peak-period preference (ratio: {peak_ratio:.1f}:1) aligns with grid needs")

    if weekend_ratio < 0.8:
        insights.append(f"Lower weekend activity suggests work-related charging patterns")

    if annual_export_per_active_vehicle > 500:
        insights.append(f"High export per active vehicle ({annual_export_per_active_vehicle:.0f} kWh/year) demonstrates significant potential")

    if capacity_factor < 0.3:
        insights.append(f"Low capacity factor ({capacity_factor:.1%}) indicates concentrated usage patterns")

    for i, insight in enumerate(insights, 1):
        print(f"   {i}. {insight}")

    if not insights:
        print("   Analysis reveals moderate V2G activity suitable for scenario modeling")

    return eda_results


# SECTION 4: PRICING DATA ANALYSIS AND MARKET CONTEXT  


In [7]:
import requests
import pandas as pd
import io
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Define SYSTEM_PARAMS if not already defined
class SYSTEM_PARAMS:
    PEAK_DEMAND_HOURS = [17, 18, 19, 20]

class PricingManager:
    """Simple pricing data manager using current API with deflator adjustment"""

    @staticmethod
    def load_current_pricing_with_deflator(region_code: str = "C", gdp_deflator: float = 1.0) -> pd.DataFrame:
        """Load current API pricing and apply GDP deflator"""
        print(f"Loading current electricity pricing for region {region_code}...")

        try:
            # Fetch current Agile pricing
            url = f"https://agilerates.uk/api/agile_rates_region_{region_code}.json"
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            data = response.json()
            rates_df = pd.json_normalize(data["rates"])

            # Process pricing data
            pricing_df = rates_df[[
                "deliveryStart",
                "agileOutgoingRate.result.rate"
            ]].rename(columns={
                "deliveryStart": "timestamp_utc",
                "agileOutgoingRate.result.rate": "export_price_p_per_kwh"
            })

            pricing_df["timestamp_utc"] = pd.to_datetime(pricing_df["timestamp_utc"], utc=True)
            pricing_df["timestamp_local"] = pricing_df["timestamp_utc"].dt.tz_convert("Europe/London")
            pricing_df["export_price_gbp_per_kwh"] = pricing_df["export_price_p_per_kwh"] / 100.0

            # Create temporal features
            pricing_df['day_of_week'] = pricing_df['timestamp_local'].dt.dayofweek
            pricing_df['hour'] = pricing_df['timestamp_local'].dt.hour

            # Create day-of-week × hour profile
            pricing_profile = pricing_df.groupby(['day_of_week', 'hour'])['export_price_gbp_per_kwh'].mean().reset_index()

            print(f"✓ API pricing loaded: {len(pricing_df)} records aggregated to {len(pricing_profile)} hourly profiles")

        except Exception as e:
            print(f"API failed: {e}")
            print("Using synthetic pricing patterns...")

            # Fallback synthetic pricing
            pricing_data = []
            for day_of_week in range(7):
                weekend_factor = 0.85 if day_of_week >= 5 else 1.0

                for hour in range(24):
                    # Time-of-use pricing pattern
                    if hour < 6:
                        base_price = 0.08      # Night
                    elif hour < 16:
                        base_price = 0.12      # Day
                    elif hour < 20:
                        base_price = 0.25      # Peak
                    else:
                        base_price = 0.15      # Evening

                    # Apply weekend discount and peak premiums
                    price = base_price * weekend_factor
                    if hour in [17, 18]:  # Peak premium
                        price *= 1.2

                    pricing_data.append({
                        'day_of_week': day_of_week,
                        'hour': hour,
                        'export_price_gbp_per_kwh': price
                    })

            pricing_profile = pd.DataFrame(pricing_data)

        # Apply GDP deflator
        pricing_profile['export_price_gbp_per_kwh'] *= gdp_deflator

        # Ensure complete 7×24 grid
        idx = pd.MultiIndex.from_product([range(7), range(24)], names=['day_of_week', 'hour'])
        pricing_profile = (pricing_profile.set_index(['day_of_week', 'hour'])
                          .reindex(idx)
                          .reset_index())
        pricing_profile['export_price_gbp_per_kwh'] = pricing_profile['export_price_gbp_per_kwh'].interpolate().bfill().ffill()

        print(f"Applied GDP deflator: {gdp_deflator:.3f}")
        print(f"Average export price: £{pricing_profile['export_price_gbp_per_kwh'].mean():.3f}/kWh")

        return pricing_profile

class ImbalancePricingManager:
    BASE_URL = "https://api.bmreports.com/BMRS/B1770/v1"

    @staticmethod
    def _session():
        s = requests.Session()
        retries = Retry(total=5, backoff_factor=0.5,
                       status_forcelist=[429,500,502,503,504],
                       allowed_methods=["GET"])
        s.mount("https://", HTTPAdapter(max_retries=retries))
        return s

    @staticmethod
    def _fetch_day_csv(api_key: str, settlement_date: str) -> pd.DataFrame:
        params = {"APIKey": api_key, "SettlementDate": settlement_date, "ServiceType": "csv"}
        r = ImbalancePricingManager._session().get(ImbalancePricingManager.BASE_URL,
                                                   params=params, timeout=60)
        r.raise_for_status()
        if not r.text or "No data" in r.text:
            return pd.DataFrame()
        df = pd.read_csv(io.StringIO(r.text))
        df.columns = [c.strip().lower().replace(" ","_") for c in df.columns]
        for col in ("system_buy_price","system_sell_price","settlement_period"):
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
        return df

    @staticmethod
    def fetch_imbalance_prices(api_key: str,
                               start_date: str = "2024-09-01",
                               end_date: str = "2024-09-15",
                               price_choice: str = "mid") -> pd.DataFrame:
        all_rows = []
        for dt in pd.date_range(start_date, end_date, freq="D"):
            sd = dt.strftime("%Y-%m-%d")
            day = ImbalancePricingManager._fetch_day_csv(api_key, sd)
            if day.empty:
                time.sleep(0.25)
                continue
            day = day.dropna(subset=["settlement_period"])
            day["settlement_period"] = day["settlement_period"].astype(int)
            day["hour"] = ((day["settlement_period"] - 1) // 2).clip(0,23)
            day["day_of_week"] = dt.dayofweek
            sbp = day.get("system_buy_price")
            ssp = day.get("system_sell_price")
            if price_choice == "sell" and sbp is not None:
                chosen = sbp
            elif price_choice == "buy" and ssp is not None:
                chosen = ssp
            else:
                chosen = (sbp + ssp) / 2.0 if sbp is not None and ssp is not None else (sbp if sbp is not None else ssp)
            day["imbalance_price_gbp_per_kwh"] = pd.to_numeric(chosen, errors="coerce") / 1000.0
            all_rows.append(day[["day_of_week","hour","imbalance_price_gbp_per_kwh"]])
            time.sleep(0.25)
        if not all_rows:
            raise RuntimeError("No BMRS imbalance data returned for the requested period.")
        raw = pd.concat(all_rows, ignore_index=True).dropna(subset=["imbalance_price_gbp_per_kwh"])
        hourly = (raw.groupby(["day_of_week","hour"], as_index=False)
                    .agg(imbalance_price_gbp_per_kwh=("imbalance_price_gbp_per_kwh","mean")))
        # Ensure 7×24 grid but keep NaNs for missing bins
        idx = pd.MultiIndex.from_product([range(7), range(24)], names=["day_of_week","hour"])
        hourly = hourly.set_index(["day_of_week","hour"]).reindex(idx).reset_index()
        print(f"✓ BMRS pulled. Rows: {len(hourly)}; NaNs: {hourly['imbalance_price_gbp_per_kwh'].isna().sum()}")
        return hourly

print("✓ PricingManager and ImbalancePricingManager classes defined")

✓ PricingManager and ImbalancePricingManager classes defined


In [8]:
def analyze_pricing_landscape() -> Dict[str, Any]:
    """
    Load and analyze your actual electricity pricing data for V2G revenue modeling
    """

    print("\n5️⃣ ELECTRICITY PRICING ANALYSIS")
    print("-" * 50)

    pricing_results = {}

    # --- Load your actual wholesale pricing data ---
    print("\n📈 5.1 Loading Your Wholesale Market Pricing")

    try:
        wholesale_pricing = PricingManager.load_current_pricing_with_deflator(
            region_code="C",
            gdp_deflator=1.0
        )

        wholesale_stats = wholesale_pricing['export_price_gbp_per_kwh'].describe()

        print(f"   ✅ Real wholesale pricing loaded!")
        print(f"     Mean price: £{wholesale_stats['mean']:.3f}/kWh")
        print(f"     Price range: £{wholesale_stats['min']:.3f} - £{wholesale_stats['max']:.3f}/kWh")

        # Peak vs off-peak analysis
        peak_avg = wholesale_pricing[wholesale_pricing['hour'].isin([17,18,19,20])]['export_price_gbp_per_kwh'].mean()
        off_peak_avg = wholesale_pricing[wholesale_pricing['hour'].isin([1,2,3,4,5])]['export_price_gbp_per_kwh'].mean()

        pricing_results['wholesale'] = {
            'source': 'Agile Rates API (Real Data)',
            'mean_price': wholesale_stats['mean'],
            'std_price': wholesale_stats['std'],
            'min_price': wholesale_stats['min'],
            'max_price': wholesale_stats['max'],
            'peak_avg': peak_avg,
            'off_peak_avg': off_peak_avg,
            'volatility_ratio': peak_avg / off_peak_avg,
            'data_points': len(wholesale_pricing)
        }

    except Exception as e:
        print(f"   ⚠️ Wholesale API failed: {e}")
        # Your fallback code here

    # --- Load your actual imbalance pricing data ---
    print("\n⚡ 5.2 Loading Your Imbalance Market Pricing")

    try:
        imbalance_pricing = ImbalancePricingManager.fetch_imbalance_prices(
            api_key='zezhzizfgqavxqh',
            start_date="2024-09-01",
            end_date="2024-09-15"
        )

        imbalance_stats = imbalance_pricing['imbalance_price_gbp_per_kwh'].describe()

        print(f"   ✅ Real ELEXON imbalance pricing loaded!")
        print(f"     Mean price: £{imbalance_stats['mean']:.3f}/kWh")

        pricing_results['imbalance'] = {
            'source': 'ELEXON BMRS API (Real Data)',
            'mean_price': imbalance_stats['mean'],
            'std_price': imbalance_stats['std'],
            'min_price': imbalance_stats['min'],
            'max_price': imbalance_stats['max'],
            'data_points': len(imbalance_pricing)
        }

    except Exception as e:
        print(f"   ⚠️ ELEXON API failed: {e}")
        # Fallback to synthetic based on wholesale

    # Market analysis
    market_premium = pricing_results['imbalance']['mean_price'] / pricing_results['wholesale']['mean_price']

    pricing_results['market_analysis'] = {
        'market_premium': market_premium
    }

    return pricing_results

# MAIN EXECUTION FUNCTION


In [9]:
def run_complete_raw_data_analysis(raw_df: pd.DataFrame, battery_capacity_kwh: float = 40.0) -> Dict[str, Any]:
    """
    Execute complete academic V2G analysis pipeline starting from raw data
    """
    print("🎓 COMPLETE ACADEMIC V2G ANALYSIS - RAW DATA TO SCENARIOS")
    print("="*80)
    print("Full pipeline from raw EV data to academic scenario analysis")
    print("="*80)

    try:
        # Section 1: Raw data validation
        print("\n" + "="*80)
        raw_validation = load_and_validate_raw_data(raw_df)

        # Section 2: V2G potential calculation (FIXED - use processed dataframe)
        print("\n" + "="*80)
        processed_df = raw_validation['processed_dataframe']  # Get the processed dataframe
        charging_sessions = calculate_v2g_potential(processed_df, battery_capacity_kwh)  # Pass the dataframe

        # Section 3: Hourly aggregation
        print("\n" + "="*80)
        hourly_df = create_hourly_aggregation(charging_sessions)

        # Section 4: Comprehensive EDA
        print("\n" + "="*80)
        eda_results = perform_comprehensive_eda(hourly_df, raw_validation)

        # Section 5: Pricing analysis
        print("\n" + "="*80)
        pricing_results = analyze_pricing_landscape()

        print("\n✅ DATA PROCESSING AND EDA COMPLETE")
        print("="*50)
        print("Processed data ready for scenario analysis")

        # Compile results package
        complete_results = {
            'raw_data_validation': raw_validation,
            'charging_sessions': charging_sessions,
            'hourly_data': hourly_df,
            'eda_results': eda_results,
            'pricing_analysis': pricing_results,
            'processing_metadata': {
                'battery_capacity_kwh': battery_capacity_kwh,
                'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M'),
                'raw_records_processed': len(raw_df),
                'hourly_records_created': len(hourly_df)
            }
        }

        return complete_results

    except Exception as e:
        print(f"\n❌ PROCESSING ERROR: {str(e)}")
        print("Please check your raw data format and required columns")
        raise

# RUN IT!

In [10]:
import pandas as pd
from datetime import datetime

# Load your data
raw_df = pd.read_csv('/content/evchargedata.csv')
print(f"Loaded {len(raw_df):,} records")

Loaded 76,698 records


In [ ]:
# Run the complete pipeline
results = run_complete_raw_data_analysis(raw_df, battery_capacity_kwh=40.0)

🎓 COMPLETE ACADEMIC V2G ANALYSIS - RAW DATA TO SCENARIOS
Full pipeline from raw EV data to academic scenario analysis


1️⃣ RAW DATA LOADING AND VALIDATION
--------------------------------------------------
📊 Raw Dataset Dimensions: 76,698 records × 5 columns
📋 Actual columns in your data:
   1. 'ParticipantID'
   2. 'BatteryChargeStartDate'
   3. 'BatteryChargeStopDate'
   4. 'Starting SoC (of 12)'
   5. 'Ending SoC (of 12)'

🔄 Column Mapping Attempt:
   ✅ 'ParticipantID' → 'vehicle_id'
   ✅ 'BatteryChargeStartDate' → 'start_time'
   ✅ 'BatteryChargeStopDate' → 'end_time'
   ✅ 'Starting SoC (of 12)' → 'start_soc_raw'
   ✅ 'Ending SoC (of 12)' → 'end_soc_raw'
✅ All required columns successfully mapped!
🚗 Vehicle Sample: 215 unique participants

🔄 Column Mapping and Standardization:
   ✓ ParticipantID → vehicle_id
   ✓ BatteryChargeStartDate → start_time
   ✓ BatteryChargeStopDate → end_time
   ✓ Starting SoC (of 12) → start_soc_raw
   ✓ Ending SoC (of 12) → end_soc_raw
🚗 Vehicle Sample

In [ ]:
# Access the results
print(f"✅ Analysis complete!")
print(f"Processed {results['processing_metadata']['raw_records_processed']:,} raw records")
print(f"Created {results['processing_metadata']['hourly_records_created']:,} hourly records")

# Access specific components
hourly_data = results['hourly_data']
eda_insights = results['eda_results']